# Conway's Game of Life (Teacher's Notes)
---

The universe of the Game of Life is an infinite two-dimensional orthogonal grid of square cells, each of which is in one of two possible states, alive or dead. Every cell interacts with its eight neighbours, which are the cells that are horizontally, vertically, or diagonally adjacent. At each step in time, the following transitions occur:

* Any live cell with fewer than two live neighbours dies, as if caused by under-population.
* Any live cell with two or three live neighbours lives on to the next generation.
* Any live cell with more than three live neighbours dies, as if by over-population.
* Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

The initial pattern constitutes the seed of the system. The first generation is created by applying the above rules simultaneously to every cell in the seed—births and deaths occur simultaneously, and the discrete moment at which this happens is sometimes called a tick (in other words, each generation is a pure function of the preceding one). The rules continue to be applied repeatedly to create further generations. [1]

## Example

Here is an example of a Game of Life board stepping through generations. The black cells are alive and the white cells are dead.

<img src="assets/gameoflife_blinker.gif",width=200,height=200>

## Design Decisions

We'll start by making some decisions about how to structure our program and the sorts of data types we'll use in our code.

> The universe of the Game of Life is an infinite two-dimensional orthogonal grid of square cells, each of which is in one of two possible states, alive or dead.

suggests that a two-dimensional array is a good structure for our Game of Life 'board'. We'll use the NumPy arrays that we learned about in the previous lesson (make sure you run the next cell).

In [1]:
import numpy as np

An important note:

> The universe of the Game of Life is an *infinite* two-dimensional orthogonal grid...

We aren't going to try to represent an infinite grid in our implementation of the Game of Life. Instead, we'll create a grid of a *finite* size and we'll imagine that the right and left edges, and the top and bottom edges wrap around to meet.

## Creating a Game of Life Board

Let's start by writing a function that will return a Game of Life board of a certain size. We'll use the number 0 to represent a dead cell, the number 1 to represent a living cell and, to start with, let's return a board with only dead cells.

In [ ]:
def create_empty_board(height, width):
    """
    This function creates an empty Game of Life game board.
    
    Parameters
    ----------
        height : integer
            the height of the board
        width : integer
            the width of the board
            
    Returns
    ----------
        board : 2d array
            a Game of Life board, with 'living' and 'dead' cells
            
    Examples
    ----------
    >>> create_empty_board(3, 3)
    array([[ 0.,  0.,  0.],
           [ 0.,  0.,  0.],
           [ 0.,  0.,  0.]])
    """
    board = np.zeros((height, width))
    return board

In [ ]:
board = create_empty_board(3, 3)
board

Now we have a Game of Life board but all the cells are dead so nothing interesting will happen. Let's write a function to create a board in which some cells are alive and some are dead. We can use the NumPy `random.choice` function to accomplish this.

Anytime you are confused about how to use a function, in Jupyter Notebook you can write the `functionname` followed by a `?` and a help box will pop up.

In [2]:
np.random.choice?

In [4]:
def create_board(height, width):
    """
    This function creates a Game of Life game board.
    
    Parameters
    ----------
        height : integer
            the height of the board
        width : integer
            the width of the board
            
    Returns
    ----------
        board : 2d array
            a Game of Life board, with 'living' and 'dead' cells
            
    Examples
    ----------
    >>> np.random.seed(1)
    >>> create_board(3, 3)
    array([[1, 1, 0],
           [0, 1, 1],
           [1, 1, 1]])
    """
    dead, alive = 0, 1
    board = np.random.choice([dead, alive], size=(height, width))
    return board

In [5]:
board = create_board(3,3)
board

array([[0, 0, 0],
       [1, 0, 1],
       [1, 0, 1]])

The way that this function currently works, each cell has a 50% chance of being alive and a 50% chance of being dead. One of the rules of Conway's Game of Life:

> Any live cell with more than three live neighbours dies, as if by over-population.

means that our game board has *way* too many living cells in it. We should add to the function so that we can control how likely a cell is to start as living.

In [24]:
def create_board(height, width, p_alive):
    """
    This function creates a Game of Life game board.
    
    Parameters
    ----------
        height : integer
            the height of the board
        width : integer
            the width of the board
        p_alive : float
            the chance that a cell is set to 'alive', must be in [0, 1]
            
    Returns
    ----------
        board : 2d array
            a Game of Life board, with 'living' and 'dead' cells
    
    Examples
    ----------
    >>> np.random.seed(1)
    >>> create_board(3, 3, .3)
    array([[0, 1, 0],           
           [0, 0, 0],
           [0, 0, 0]])
    """
    dead, alive = 0, 1
    p_dead = 1 - p_alive # We know that if a cell is not alive, it must be dead
    board = np.random.choice([dead, alive], p=[p_dead, p_alive], size=(height, width))
    
    return board

In [25]:
board = create_board(3, 3, p_alive=0)
board

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [26]:
board = create_board(3, 3, p_alive=.2)
board

array([[0, 0, 0],
       [1, 1, 0],
       [0, 0, 0]])

In [27]:
board = create_board(3, 3, p_alive=1)
board

array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]])

## Scripting the Simulation

### Pseudocode

Writing 'pseudocode' is a great first step for writing complicated programs. Pseudocode is a program written in plain english that describes --- at a high level --- what your program is doing. Because it isn't meant to be run, you don't have to deal with any bugs. Here's an example of some pseudocode describing an algorithm for how the Game of Life simulation steps along to its next generation:

    Create a copy of the current board for the next generation board
    For each cell in the current board:
        Count the number of living neighbors it has.
        Compute the state of the cell in the next generation
        Set the cell's value in the next board to either alive or dead.
        
Now that we have a pseudocode skeleton, we can make some design decisions about how we'll implement this in actual code. Let's code up this algorithm in a function called `step_forward()`. 

One good practice in programming is to split up complex functions into smaller, less-complex functions. If this whole function is `step_forward()`, let's look for other logical groups of code that we could split out and write as separate functions.

This bit:

    Count the number of living neighbors a cell has
    
Is complex enough that we could write it in a separate function, `count_living_neighbors()`.

Another bit of code that we could split out:

    Compute the state of this cell in the next generation
        
Into a function called `compute_next_state()`.

### Python Code
Let's start with the `count_living_neighbors()` function. Given some cell, we need to count the number of its 8 neighbors that are alive. This one can be tricky because of the way that Python handles array indexing. Let's say we write the following function:

In [30]:
def count_living_neighbors(board, r, c):
    """
    Counts the number of living neighbors around an array cell.
    
    Parameters
    ----------
        board : 2d array
        r : int
            row index
        c : int
            column index
            
    Returns
    ----------
        int
            number of living neighbors

    """
    u_ = board[r-1, c] # Move up a row
    d_ = board[r+1, c] # Move down a row
    l_ = board[r, c-1] # Move left a column
    r_ = board[r, c+1] # etc...
    ul = board[r-1, c-1]
    ur = board[r-1, c+1]
    dl = board[r+1, c-1]
    dr = board[r+1, c+1]
    
    # Remember that we are representing the states of the cells with 
    # the numbers 1 (alive) and 0 (dead), so if we add the values
    # in each cell, we'll end up with the tally of living cells.
    return (u_ + d_ + l_ + r_ + ul + ur + dl + dr)

Let's test this function out on the following Game of Life board:

In [31]:
board = np.array([
    [0, 1, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 1],
    [1, 0, 0, 0]
])

Counting the number of neighbors for some of the middle elements works as expected:

In [32]:
count_living_neighbors(board, 1, 1) == 2

True

In [33]:
count_living_neighbors(board, 1, 2) == 4

True

But if we try to do it on an edge:

In [34]:
count_living_neighbors(board, 0, 3) == 1

IndexError: index 4 is out of bounds for axis 1 with size 4

we get the error:

    IndexError: index 4 is out of bounds for axis 1 with size 4
    
How do we fix this? Let's remember that Python uses 0-based indexing. An array with 4 elements has indices 0, 1, 2, and 3. Trying to access the array entry at position 4 raises an IndexError because there *is* no entry at position 4. 

* NOTE: Python supports negative indexing, so we only encounter this problem on the right and bottom edges of the array. For example, the indices -1, -2, -3, and -4 are valid for an array with 4 elements. These are reckoned backwards from the last entry in the array, so -1 is the last element, -2 is the second-to-last, etc...

A good question to ask:

> What behavior do we want our program to have when it tries to access values outside of the array? 

We've decided that we want our board to 'wrap around' (e.g. the first element outside of the right side of a row should be the first element on the left side of the same row). In this specific case, for example, when we try to access the element at position 4, we really want the element at position 0.

Enter the % (modulo) operator.

### Modulo

> The modulo operation finds the remainder after division of one number by another. [2]

Just like we use the `*` operator for multiplication and the `+` operator for addition, we can use the `%` operator for the modulo operation. Here are some examples:

In [35]:
# 1 divides into 1 with no remainder.
1 % 1

0

In [36]:
# 1 divides into 2 with no remainder.
2 % 1

0

In [38]:
# 2 divides into 5 with a remainder of 1.
5 % 2

1

So how can we apply this to array indices?

Well, let's say we have the following 4-element array:

In [41]:
array = [0, 1, 2, 3]
print('Array:', array)
print('Length:', len(array))

Array: [0, 1, 2, 3]
Length: 4


If we try to access the element at position 4, like we did earlier:

In [42]:
i = 4
array[i]

IndexError: list index out of range

We get an error. Let's try applying the modulo operator in this situation:

In [44]:
i = 4
i % len(array)

0

So if we modulo a number by the length of an array, we'll always get a valid position in such a way that gives us our desired, 'wrap-around' behavior.

Now, let's try re-writing the `count_living_neighbors` function using the % operator to avoid getting an IndexError.

In [47]:
def count_living_neighbors(board, r, c):
    """
    Counts the number of living neighbors around an array cell.
    
    Parameters
    ----------
        board : 2d array
        r : int
            row index
        c : int
            column index
            
    Returns
    ----------
        int
            number of living neighbors

    """
    num_rows, num_cols = board.shape
    
    u_ = board[(r-1) % num_rows, c % num_cols]
    d_ = board[(r+1) % num_rows, c % num_cols]
    l_ = board[r % num_rows, (c-1) % num_cols]
    r_ = board[r % num_rows, (c+1) % num_cols]
    ul = board[(r-1) % num_rows, (c-1) % num_cols]
    ur = board[(r-1) % num_rows, (c+1) % num_cols]
    dl = board[(r+1) % num_rows, (c-1) % num_cols]
    dr = board[(r+1) % num_rows, (c+1) % num_cols]
    
    return (u_ + d_ + l_ + r_ + ul + ur + dl + dr)

## Compute Next State
A reminder of the rules of the Game of Life:

* Any live cell with fewer than two live neighbours dies, as if caused by under-population.
* Any live cell with two or three live neighbours lives on to the next generation.
* Any live cell with more than three live neighbours dies, as if by over-population.
* Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

Let's start by detailing this function with some pseudocode:

        If (the number of living neighbors < 2) and (the cell is alive):
            It will be dead in the next board
        If (the number of living neighbors = 2 or 3) and (the cell is alive):
            It will be alive in the next board
        If (the number of living neighbors > 3) and the cell is alive:
            It will be dead in the next board
        If (the number of living neighbors = 3) and (the cell is dead):
            It will be alive in the next board

In [48]:
def compute_next_state(current_state, num_living_neighbors):
    """
    Computes the state of a cell in the next generation of a Game of Life.
    
    Parameters
    ----------
        current_state : int
        num_living_neighbors : int
        
    Returns
    ----------
        next_state : int
        
    """
    dead, alive = 0, 1
    next_state = dead
    
    if (num_living_neighbors < 2) and (current_state == alive):
        # Dies by under-population
        next_state = dead
    elif ( 2 <= num_living_neighbors <= 3) and (current_state == alive):
        # Survives on to the next generation
        next_state = alive
    elif (num_living_neighbors > 3) and (current_state == alive):
        # Dies by over-population
        next_state = dead
    elif (num_living_neighbors == 3) and (current_state == dead):
        # Live cell is birthed 
        next_state = alive
    
    return next_state

## Putting It All Together

Here's the pseudocode we wrote earlier:
    
    Create a copy of the current board for the next generation board
    For each cell in the current board:
        Count the number of living neighbors it has.
        Compute the state of the cell in the next generation
        Set the cell's value in the next board to either alive or dead.

In [49]:
def step_forward(board):
    """
    Computes the next generation of a Game of Life board.
    
    Parameters
    ----------
        board : 2d array
        
    Returns
    ----------
        next_board : 2d array
    """
    # Copy the current board for the next generation
    next_board = board.copy()
    # Loop over all of the cells in the current board
    rows, cols = board.shape
    for r in range(rows):
        for c in range(cols):
            # Fetch the current state of this cell
            current_state = board[r, c]
            # Count the number of living neighbors
            num_living_neighbors = count_living_neighbors(board, r, c)
            # Figure out if this cell will be alive or dead in the next board
            next_state = compute_next_state(current_state, num_living_neighbors)
            # Set the state of this cell in the next board
            next_board[r, c] = next_state  
    
    return next_board

## Testing it Out

Let's try it out!

In [72]:
# Create a new board
board = create_board(5, 5, .3)
print('Starting:')
print(board)

# Step the board forward
print('Next Step:')
board = step_forward(board)
print(board)

Starting:
[[0 0 1 0 1]
 [0 1 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]
 [0 0 0 0 0]]
Next Step:
[[0 1 1 1 0]
 [0 1 1 0 0]
 [0 0 1 1 0]
 [0 0 0 0 0]
 [0 0 0 1 0]]


## References

1. https://en.wikipedia.org/wiki/Conway's_Game_of_Life

2. https://en.wikipedia.org/wiki/Modulo_operation